In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertForPreTraining
import torch

**BERT Model can be finetuned using NSP OR MLM OR Both**

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForPreTraining.from_pretrained('bert-base-uncased')

#Prediction of Masked Words
#Masked words are elections and attacked

text = ("After Abraham Lincoln won the November 1860 presidential [MASK] on an "
        "anti-slavery platform, an initial seven slave states declared their "
        "secession from the country to form the Confederacy. War broke out in "
        "April 1861 when secessionist forces [MASK] Fort Sumter in South "
        "Carolina, just over a month after Lincoln's inauguration.")



inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
outputs.keys()

odict_keys(['prediction_logits', 'seq_relationship_logits'])

In [ ]:
outputs.prediction_logits

tensor([[[ -7.6192,  -7.5433,  -7.6124,  ...,  -6.7155,  -6.7375,  -4.6122],
         [-12.5489, -12.3772, -12.6500,  ..., -11.8643, -11.4446,  -9.1151],
         [ -6.2346,  -6.3590,  -5.9091,  ...,  -6.1258,  -6.2720,  -5.0268],
         ...,
         [ -2.2497,  -2.1352,  -2.1812,  ...,  -1.7201,  -1.2728,  -7.8301],
         [-14.2654, -14.3100, -14.2294,  ..., -11.4669, -11.7212, -10.3129],
         [-11.5071, -12.0389, -11.6046,  ..., -11.2875,  -9.1655,  -9.1732]]],
       grad_fn=<ViewBackward0>)

In [ ]:
#For 1st text it contains 62 tokens and vocab size is 30522
outputs.prediction_logits.shape
#output=outputs.seq_relationship_logits
#outputs.prediction_logits is the output from the MLM head (vocab which maps to a word from the vocab after softmax)
#The prediction_logits represent the raw scores (logits) for each token in the vocabulary.
#These scores indicate how likely each token is to be the correct prediction for the masked position.


torch.Size([1, 62, 30522])

In [ ]:
#prob for 2nd token in 1st text with respect to words in vocab size
outputs.prediction_logits[0][1]


tensor([-12.5489, -12.3772, -12.6500,  ..., -11.8643, -11.4446,  -9.1151],
       grad_fn=<SelectBackward0>)

In [ ]:
#prob for 1st token in 1st text with respect to words in vocab size
outputs.prediction_logits[0][0]


tensor([-7.6192, -7.5433, -7.6124,  ..., -6.7155, -6.7375, -4.6122],
       grad_fn=<SelectBackward0>)

In [ ]:
len(outputs.prediction_logits[0][1])

30522

In [ ]:
token2idx = tokenizer.get_vocab()

In [ ]:
token2idx['hello']

7592

In [ ]:
#Interchanging key & value
#Index to token
idx2token = {value:key for key, value in token2idx.items()}

In [ ]:
idx2token[7592]

'hello'

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])

torch.sum(mat, dim=-1)

tensor([ 3, 12, 21])

If it is 0 then column wise

In [ ]:
outputs.prediction_logits[0][1]

tensor([-12.5489, -12.3772, -12.6500,  ..., -11.8643, -11.4446,  -9.1151],
       grad_fn=<SelectBackward0>)

In [ ]:
softmax = torch.nn.functional.softmax(outputs.prediction_logits[0][0], dim=-1)  # create probability distribution
#This is done for 1st token

In [ ]:
softmax1 = torch.nn.functional.softmax(outputs.prediction_logits[0][1],dim=-1) #second token

In [ ]:
softmax1

tensor([1.9345e-16, 2.2970e-16, 1.7485e-16,  ..., 3.8360e-16, 5.8366e-16,
        5.9959e-15], grad_fn=<SoftmaxBackward0>)

In [ ]:
softmax

tensor([2.0750e-07, 2.2385e-07, 2.0891e-07,  ..., 5.1223e-07, 5.0109e-07,
        4.1967e-06], grad_fn=<SoftmaxBackward0>)

In [ ]:
len(softmax)

30522

In [ ]:
argmax = torch.argmax(softmax)
argmax

tensor(1012)

In [ ]:
idx2token[2044],idx2token[1012]

('after', '.')

In [ ]:
softmax2=torch.nn.functional.softmax(outputs.prediction_logits[0][2],dim=-1)
argmax2 = torch.argmax(softmax2)
print(argmax2)
idx2token[8181]


tensor(8181)


'abraham'

In [ ]:
#Done for overall tokens
softmax = torch.nn.functional.softmax(outputs.prediction_logits[0], dim=-1)  # create probability distribution for 1st sentence
argmax = torch.argmax(softmax,dim=1)  # get index of the max probability
argmax

tensor([ 1012,  2044,  8181,  5367,  2180,  1996,  2281,  7313,  4883,  2602,
         2006,  2019,  3424,  1011,  8864,  4132,  1010,  2019,  3988,  2698,
         6658,  2163,  4161,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
         1996, 18179,  1012,  2162,  3631,  2041,  1999,  2258,  6863,  2043,
        22965,  2923,  2749,  4110,  3481,  7680, 10907,  1999,  2148,  3792,
         1010,  2074,  2058,  1037,  3204,  2044,  5367,  1005,  1055, 17331,
         1012,  1055])

In [ ]:
softmax

tensor([[2.0750e-07, 2.2385e-07, 2.0891e-07,  ..., 5.1223e-07, 5.0109e-07,
         4.1967e-06],
        [1.9345e-16, 2.2970e-16, 1.7485e-16,  ..., 3.8360e-16, 5.8366e-16,
         5.9959e-15],
        [4.6866e-12, 4.1385e-12, 6.4900e-12,  ..., 5.2254e-12, 4.5147e-12,
         1.5682e-11],
        ...,
        [2.9416e-12, 3.2985e-12, 3.1500e-12,  ..., 4.9952e-12, 7.8132e-12,
         1.1092e-14],
        [5.2495e-17, 5.0201e-17, 5.4415e-17,  ..., 8.6191e-16, 6.6839e-16,
         2.7331e-15],
        [1.0184e-09, 5.9838e-10, 9.2383e-10,  ..., 1.2685e-09, 1.0590e-08,
         1.0508e-08]], grad_fn=<SoftmaxBackward0>)

In [ ]:
len(argmax)

62

In [ ]:
for i in argmax:
    print(idx2token[i.item()], end=' ')

. after abraham lincoln won the november 1860 presidential election on an anti - slavery platform , an initial seven slave states declared their secession from the country to form the confederacy . war broke out in april 1861 when secession ##ist forces captured fort sum charleston in south carolina , just over a month after lincoln ' s inauguration . s 

In [ ]:
text

"After Abraham Lincoln won the November 1860 presidential [MASK] on an anti-slavery platform, an initial seven slave states declared their secession from the country to form the Confederacy. War broke out in April 1861 when secessionist forces [MASK] Fort Sumter in South Carolina, just over a month after Lincoln's inauguration."

**MLM Training Logic**

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

text = ("After Abraham Lincoln won the November 1860 presidential [MASK] on an "
        "anti-slavery platform, an initial seven slave states declared their "
        "secession from the country to form the Confederacy. War broke out in "
        "April 1861 when secessionist forces [MASK] Fort Sumter in South "
        "Carolina, just over a month after Lincoln's inauguration.")

# No padding
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
outputs.keys()

odict_keys(['logits'])

In [ ]:
outputs.logits.shape

torch.Size([1, 62, 30522])

In [ ]:
mask_pos = torch.flatten((inputs.input_ids[0] == 103).nonzero()).tolist()
mask_pos

[9, 43]

In [ ]:
inputs.input_ids[0]

tensor([  101,  2044,  8181,  5367,  2180,  1996,  2281,  7313,  4883,   103,
         2006,  2019,  3424,  1011,  8864,  4132,  1010,  2019,  3988,  2698,
         6658,  2163,  4161,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
         1996, 18179,  1012,  2162,  3631,  2041,  1999,  2258,  6863,  2043,
        22965,  2923,  2749,   103,  3481,  7680,  3334,  1999,  2148,  3792,
         1010,  2074,  2058,  1037,  3204,  2044,  5367,  1005,  1055, 17331,
         1012,   102])

In [ ]:
inputs.input_ids[0][1]

tensor(2044)

In [ ]:
inputs
#token type ids=0 indicates 1st sentence. 1 indicates 2nd sentence and so on. There is only one sentence.
#attention mask=1, as there is no  real tokens and padded words

{'input_ids': tensor([[  101,  2044,  8181,  5367,  2180,  1996,  2281,  7313,  4883,   103,
          2006,  2019,  3424,  1011,  8864,  4132,  1010,  2019,  3988,  2698,
          6658,  2163,  4161,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
          1996, 18179,  1012,  2162,  3631,  2041,  1999,  2258,  6863,  2043,
         22965,  2923,  2749,   103,  3481,  7680,  3334,  1999,  2148,  3792,
          1010,  2074,  2058,  1037,  3204,  2044,  5367,  1005,  1055, 17331,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
rand = torch.rand(inputs.input_ids.shape)
rand

tensor([[0.0056, 0.3965, 0.1281, 0.8858, 0.6067, 0.2831, 0.9708, 0.3715, 0.4973,
         0.0512, 0.5793, 0.7394, 0.0783, 0.9820, 0.2906, 0.7346, 0.8577, 0.9666,
         0.5698, 0.2980, 0.4978, 0.5265, 0.8096, 0.9392, 0.3025, 0.9429, 0.4747,
         0.5685, 0.7716, 0.9931, 0.5992, 0.6465, 0.9293, 0.3790, 0.7266, 0.2502,
         0.2909, 0.2727, 0.8404, 0.6145, 0.9021, 0.2531, 0.9048, 0.7488, 0.2708,
         0.1798, 0.6261, 0.8340, 0.7529, 0.3934, 0.9438, 0.0962, 0.1947, 0.2241,
         0.0584, 0.6361, 0.3930, 0.8662, 0.3911, 0.0706, 0.4512, 0.4499]])

In [ ]:
#Dont mask the tokens if it is 101 or 102
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * (inputs.input_ids != 102)
mask_arr

tensor([[False, False,  True, False, False, False, False, False, False,  True,
         False, False,  True, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False,  True, False, False,  True, False, False, False, False,  True,
         False, False]])

In [ ]:
selection = torch.flatten((mask_arr[0]).nonzero()).tolist()
selection

[2, 9, 12, 51, 54, 59]

In [ ]:
len(selection)

6

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()
inputs

{'input_ids': tensor([[  101,  2044,  8181,  5367,  2180,  1996,  2281,  7313,  4883,   103,
          2006,  2019,  3424,  1011,  8864,  4132,  1010,  2019,  3988,  2698,
          6658,  2163,  4161,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
          1996, 18179,  1012,  2162,  3631,  2041,  1999,  2258,  6863,  2043,
         22965,  2923,  2749,   103,  3481,  7680,  3334,  1999,  2148,  3792,
          1010,  2074,  2058,  1037,  3204,  2044,  5367,  1005,  1055, 17331,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([

**Let's take a dataset and train BERT on MLM**

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
texts = [
    "The ebullient sun danced across the azure sky.",
    "Amidst the bustling city, a serene park beckoned.",
    "His laughter echoed through the cobblestone streets.",
    "The ancient tome revealed secrets of forgotten civilizations.",
    "She sipped her tea, lost in the labyrinth of her thoughts.",
    "The moon whispered secrets to the silvery waves.",
    "In the twilight, shadows wove a mysterious tapestry.",
    "The old oak stood as a sentinel of time.",
    "The aroma of freshly baked bread filled the cozy boulangerie.",
    "Beneath the star-studded canvas, dreams took flight.",
    "The whimsical carousel spun its colorful tales.",
    "Lost in the verdant forest, she found solace.",
    "The melancholy notes of a violin echoed.",
    "The effervescent stream flowed over smooth stones.",
    "His eyes held the constellations of distant galaxies.",
    "The enigmatic door led to realms unknown.",
    "The crimson leaves whispered autumn's arrival.",
    "The ethereal mist veiled the ancient ruins.",
    "Her laughter was a symphony in the quiet room.",
    "The iridescent butterfly danced on sunbeams.",
    "The time-worn map revealed hidden treasures.",
    "The whispering wind carried secrets from afar.",
    "The gossamer wings of dragonflies shimmered.",
    "The quixotic quest led to a mythical fountain.",
    "The effulgent sunrise painted the horizon.",
    "Amidst the cacophony of the city, he found peace.",
    "The halcyon days of summer lingered.",
    "The ephemeral beauty of cherry blossoms.",
    "His smile held the warmth of a thousand suns.",
    "The resplendent peacock displayed its plumage.",
    "The luminous moon guided sailors home.",
    "The whirlwind of emotions swept her away.",
    "The sapphire sea stretched to infinity.",
    "The effervescent laughter of children echoed.",
    "The enchanting forest whispered ancient tales.",
    "The serendipitous encounter changed everything.",
    "The incandescent fireflies danced in the night.",
    "The ephemeral snowflakes kissed the ground.",
    "His heart beat in syncopation with hers.",
    "The resolute climber reached the summit."
]




In [ ]:
texts[:5]

['The ebullient sun danced across the azure sky.',
 'Amidst the bustling city, a serene park beckoned.',
 'His laughter echoed through the cobblestone streets.',
 'The ancient tome revealed secrets of forgotten civilizations.',
 'She sipped her tea, lost in the labyrinth of her thoughts.']

In [ ]:
inputs = tokenizer(texts, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
inputs

{'input_ids': tensor([[  101,  1996,  1041,  ...,     0,     0,     0],
        [  101, 17171,  1996,  ...,     0,     0,     0],
        [  101,  2010,  7239,  ...,     0,     0,     0],
        ...,
        [  101,  1996,  4958,  ...,     0,     0,     0],
        [  101,  2010,  2540,  ...,     0,     0,     0],
        [  101,  1996, 24501,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()
inputs

{'input_ids': tensor([[  101,  1996,  1041,  ...,     0,     0,     0],
        [  101, 17171,  1996,  ...,     0,     0,     0],
        [  101,  2010,  7239,  ...,     0,     0,     0],
        ...,
        [  101,  1996,  4958,  ...,     0,     0,     0],
        [  101,  2010,  2540,  ...,     0,     0,     0],
        [  101,  1996, 24501,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[  101,  1996,  1041,  ...,     0,     0,     0],
        [  101, 17171,  1996,  ...,     0,     0,     0],
        [  101,  2010, 

In [ ]:
# we have 40 texts and 512 tokens
print(len(inputs.input_ids[0]))
print(len(inputs.input_ids)) #40 texts

512
40


In [ ]:
inputs.input_ids.shape

torch.Size([40, 512])

In [ ]:

rand = torch.rand(inputs.input_ids.shape)
# avoid cls token, sep token, pad token
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [ ]:
mask_arr.shape

torch.Size([40, 512])

In [ ]:
selection = []
for i in range(inputs.input_ids.shape[0]):
    selection.append(torch.flatten(mask_arr[i].nonzero()).tolist())

In [ ]:
print(len(selection))
selection

40


[[],
 [3, 9],
 [4, 10],
 [5],
 [6, 8, 9],
 [],
 [1, 7, 9],
 [],
 [4],
 [6],
 [3],
 [3],
 [],
 [1, 3, 5, 7, 8],
 [],
 [4, 7],
 [5],
 [3, 6],
 [],
 [2],
 [4, 6, 9],
 [2, 7],
 [6, 7, 12],
 [],
 [7],
 [8, 9],
 [1, 8],
 [3, 5],
 [6, 8, 10],
 [],
 [2, 4],
 [11],
 [3],
 [6, 9],
 [2, 4],
 [7],
 [1, 6, 8],
 [3, 4],
 [4],
 [10]]

In [ ]:
inputs.input_ids[0,1] #0=1st text, 1=2nd token in 1st text


tensor(1996)

In [ ]:
#Replace all masked tokens with 103
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103



In [ ]:
for key, val in inputs.items():
    print(key)

input_ids
token_type_ids
attention_mask
labels


In [ ]:
#encodings=inputs
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = Dataset(inputs)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True)

In [ ]:
cuda_available = torch.cuda.is_available()

In [ ]:
cuda_available

True

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
print(model.device)

cuda:0


In [ ]:
#Adam with weighted decay - reduces chance of overfitting
from transformers import AdamW
model.train()
optim = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from tqdm import tqdm  # for our progress bar

epochs = 20

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/5 [00:00<?, ?it/s]<ipython-input-15-24730ae54740>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 19: 100%|██████████| 5/5 [00:01<00:00,  3.54it/s, loss=0.0202]


**Fine-Tuned Model Combined With BertForMaskedLM**

In [ ]:
torch.save(model, 'my_trained_model.pth')

In [ ]:
fine_tuned_model = torch.load('my_trained_model.pth')

In [ ]:
bert_masked_lm = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
bert_masked_lm.load_state_dict(fine_tuned_model.state_dict())

<All keys matched successfully>

In [ ]:
text = ("Amidst the bustling city, a serene [MASK] beckoned")

inputs = tokenizer(text, return_tensors="pt")
outputs = bert_masked_lm(**inputs)

In [ ]:
outputs.logits

tensor([[[ -5.0880, -10.8094, -11.1884,  ..., -10.1558, -13.0333,  -8.6899],
         [  7.3000,   0.9649,   0.2174,  ...,  -0.4030,  -2.3461,   2.8378],
         [-10.7196, -13.9102, -13.9742,  ..., -11.5989, -13.9253, -11.4464],
         ...,
         [  0.8023,  -3.9212,  -4.5457,  ...,  -4.5887,  -5.7461,  -4.1787],
         [  1.3501,  -3.5770,  -4.0456,  ...,  -3.4686,  -4.6876,  -5.5508],
         [ -5.1164, -10.2441, -10.2301,  ...,  -9.7744, -10.8149,  -9.3448]]],
       grad_fn=<ViewBackward0>)

In [ ]:
softmax = torch.nn.functional.softmax(outputs.logits[0], dim=-1)
argmax = torch.argmax(softmax,dim=1)
argmax

tensor([  101, 17171,  1996, 13950,  2989,  2103,  1010,  1037, 25388,  2380,
        10272, 17799,   102])

In [ ]:
token2idx = tokenizer.get_vocab()

In [ ]:
idx2token = {value:key for key, value in token2idx.items()}

In [ ]:
for i in argmax:
    print(idx2token[i.item()], end=' ')

[CLS] amidst the bust ##ling city , a serene park beck ##oned [SEP] 

In [ ]:
outputs.keys()

odict_keys(['logits'])